# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [8]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import matplotlib.pyplot as plt

# Import API key
from api_keys import geoapify_key

In [9]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,west island,-12.1568,96.8225,25.99,83,100,3.60,CC,1701731960
1,1,kudahuvadhoo,2.6708,72.8944,26.97,80,44,5.20,MV,1701731516
2,2,hamilton,39.1834,-84.5333,6.35,81,75,3.60,US,1701731654
3,3,port-aux-francais,-49.3500,70.2167,4.10,73,80,11.68,TF,1701731948
4,4,iskateley,67.6803,53.1512,-26.10,93,20,4.02,RU,1701731959


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [10]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 1000,
    frame_height = 500,
    size = "Humidity",
    scale = 0.9,
    hover_cols=["City"],
    line_color="black", 
    color = "City" 
)


# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [11]:
# Narrow down cities that fit criteria and drop any results with null values
indeal_conditions_df = city_data_df[(city_data_df['Max Temp'] < 27) & (city_data_df['Max Temp'] > 21) & (city_data_df['Cloudiness']== 0) & (city_data_df["Wind Speed"] < 4.5)]


# Drop any rows with null values
clean_data_df = indeal_conditions_df.dropna()

# Display sample data
clean_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
171,171,cuatro cienegas de carranza,26.9833,-102.0833,21.20,25,0,2.95,MX,1701732071
191,191,dahab,28.5010,34.5134,24.39,49,0,1.22,EG,1701731778
269,269,quelimane,-17.8786,36.8883,22.31,82,0,1.62,MZ,1701732105
418,418,kidal,18.4411,1.4078,22.96,11,0,3.33,ML,1701731781
456,456,seaside,36.6111,-121.8516,21.19,67,0,3.13,US,1701732050
463,463,warin chamrap,15.1932,104.8628,23.90,88,0,2.57,TH,1701732183


### Step 3: Create a new DataFrame called `hotel_df`.

In [12]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = clean_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
171,cuatro cienegas de carranza,MX,26.9833,-102.0833,25,
191,dahab,EG,28.5010,34.5134,49,
269,quelimane,MZ,-17.8786,36.8883,82,
418,kidal,ML,18.4411,1.4078,11,
456,seaside,US,36.6111,-121.8516,67,
463,warin chamrap,TH,15.1932,104.8628,88,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [13]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "type": "accommodation.hotel", 
    "limit":20,
    "apiKey":geoapify_key    
}



# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    
    latitude = row["Lat"]
    longitude = row["Lng"]
    
   

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
cuatro cienegas de carranza - nearest hotel: No hotel found
dahab - nearest hotel: No hotel found
quelimane - nearest hotel: No hotel found
kidal - nearest hotel: No hotel found
seaside - nearest hotel: No hotel found
warin chamrap - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
171,cuatro cienegas de carranza,MX,26.9833,-102.0833,25,No hotel found
191,dahab,EG,28.5010,34.5134,49,No hotel found
269,quelimane,MZ,-17.8786,36.8883,82,No hotel found
418,kidal,ML,18.4411,1.4078,11,No hotel found
456,seaside,US,36.6111,-121.8516,67,No hotel found
463,warin chamrap,TH,15.1932,104.8628,88,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 1000,
    frame_height = 500,
    size = "Humidity",
    scale = 0.9,
    hover_cols=["City","Hotel Name","Country"],
    line_color="black", 
    color = "City"
)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)